In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier

In [2]:
input_data = pd.read_csv("train.csv", index_col='PassengerId')

def set_age(name):
    if(name.find('Mr.') != -1): #male - married or not
        return 30
    elif(name.find('Mrs.') != -1): #female - married
        return 45
    elif(name.find('Ms.') != -1): #female - married or not 
        return 30
    elif(name.find('Miss.') != -1): #female - not married 
        return 20
    else:
        return 45
    
max_fare = input_data['Fare'].max()
min_fare = input_data['Fare'].min()
print(max_fare, min_fare)

def clean_dataset(dataset):
    popular_port = dataset['Embarked'].dropna().mode()[0] 
    print("This port is most popular and replace all 'NA' to " + popular_port)
    dataset['Embarked'] = dataset['Embarked'].fillna(popular_port) 
    dataset['Companions'] = dataset['SibSp'] + dataset['Parch'] #number of people travelling with
    dataset['Female'] = dataset['Sex'].map({'female': 1, 'male': 0})
    dataset['Male'] = dataset['Sex'].map({'female': 0, 'male': 1})
    #Embarks as one-hot
    embarked_one_hot = pd.get_dummies(dataset['Embarked'], prefix='Embarked') 
    dataset = dataset.join(embarked_one_hot) 
    #Fare intervals -> one-hot
    fares_intervals = pd.cut(x=dataset['Fare'], bins=[0, 50, 100, 200, 500, 1000])
    fares_one_hot = pd.get_dummies(fares_intervals, prefix='Fare')
    dataset = dataset.join(fares_one_hot)
    dataset['Age'] = dataset.apply(
                            lambda row: set_age(row['Name']) if np.isnan(row['Age']) else row['Age'], axis=1)
    
    dataset = dataset.drop(['Cabin', 'Ticket', 'Name', 'SibSp', 'Parch', 'Embarked', 'Sex', 'Fare'], axis=1) #drop useless columns
    return(dataset)
    

train_data = clean_dataset(input_data)
train_data.to_csv('train_data_results.csv', index=False)
print("Prepared data:")
train_data

512.3292 0.0
This port is most popular and replace all 'NA' to S
Prepared data:


,Survived,Pclass,Age,Companions,Female,Male,Embarked_C,Embarked_Q,Embarked_S,"Fare_(0, 50]","Fare_(50, 100]","Fare_(100, 200]","Fare_(200, 500]","Fare_(500, 1000]"
PassengerId,,,,,,,,,,,,,,
1,0,3,22.0,1,0,1,0,0,1,1,0,0,0,0
2,1,1,38.0,1,1,0,1,0,0,0,1,0,0,0
3,1,3,26.0,0,1,0,0,0,1,1,0,0,0,0
4,1,1,35.0,1,1,0,0,0,1,0,1,0,0,0
5,0,3,35.0,0,0,1,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,27.0,0,0,1,0,0,1,1,0,0,0,0
888,1,1,19.0,0,1,0,0,0,1,1,0,0,0,0
889,0,3,20.0,3,1,0,0,0,1,1,0,0,0,0


In [3]:
 # X and Y ('survived;) axis
 X = train_data.drop(['Survived'], axis=1).values.astype(float)

 Y = train_data['Survived'].values

 def create_model(optimizer='adam', init='uniform'):
    #create model
    model = keras.Sequential()
    model.add( keras.layers.Dense(16, input_dim=X.shape[1], kernel_initializer=init, activation='relu') )
    model.add( keras.layers.Dense(8, kernel_initializer=init, activation='relu') ) 
    model.add( keras.layers.Dense(4, kernel_initializer=init, activation='relu') ) 
    model.add( keras.layers.Dense(1, kernel_initializer=init, activation='sigmoid') )
    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print("Model created")
    return model

In [4]:
epochs = 200
batch_size = 5
init = 'glorot_uniform'
optimizer = 'rmsprop'

#create classfier
model_predictions = KerasClassifier(
    model=create_model, optimizer=optimizer, init=init, epochs=epochs,
    batch_size=batch_size)

model_predictions.fit(X,Y)

Model created
Epoch 1/200
179/179 [==============================] - 2s 3ms/step - loss: 0.7361 - accuracy: 0.5735
Epoch 2/200
179/179 [==============================] - 1s 5ms/step - loss: 0.6534 - accuracy: 0.6184
Epoch 3/200
179/179 [==============================] - 1s 3ms/step - loss: 0.6359 - accuracy: 0.6274
Epoch 4/200
179/179 [==============================] - 1s 3ms/step - loss: 0.6040 - accuracy: 0.6543
Epoch 5/200
179/179 [==============================] - 1s 3ms/step - loss: 0.5671 - accuracy: 0.7149
Epoch 6/200
179/179 [==============================] - 1s 3ms/step - loss: 0.5259 - accuracy: 0.7452
Epoch 7/200
179/179 [==============================] - 1s 3ms/step - loss: 0.4892 - accuracy: 0.7845
Epoch 8/200
179/179 [==============================] - 1s 3ms/step - loss: 0.4704 - accuracy: 0.7890
Epoch 9/200
179/179 [==============================] - 1s 3ms/step - loss: 0.4569 - accuracy: 0.7879
Epoch 10/200
179/179 [==============================] - 1s 3ms/step - loss: 0

179/179 [==============================] - 0s 3ms/step - loss: 0.3910 - accuracy: 0.8316
Epoch 82/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3987 - accuracy: 0.8260
Epoch 83/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3910 - accuracy: 0.8350
Epoch 84/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3871 - accuracy: 0.8406
Epoch 85/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3889 - accuracy: 0.8328
Epoch 86/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3913 - accuracy: 0.8283
Epoch 87/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3980 - accuracy: 0.8249
Epoch 88/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3865 - accuracy: 0.8440
Epoch 89/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3872 - accuracy: 0.8328
Epoch 90/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3899 - accuracy: 

179/179 [==============================] - 0s 3ms/step - loss: 0.3730 - accuracy: 0.8350
Epoch 161/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3762 - accuracy: 0.8328
Epoch 162/200
179/179 [==============================] - 0s 2ms/step - loss: 0.3766 - accuracy: 0.8316
Epoch 163/200
179/179 [==============================] - 1s 3ms/step - loss: 0.3703 - accuracy: 0.8373
Epoch 164/200
179/179 [==============================] - 1s 3ms/step - loss: 0.3733 - accuracy: 0.8440
Epoch 165/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3765 - accuracy: 0.8339
Epoch 166/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3708 - accuracy: 0.8294
Epoch 167/200
179/179 [==============================] - 0s 2ms/step - loss: 0.3698 - accuracy: 0.8361
Epoch 168/200
179/179 [==============================] - 0s 2ms/step - loss: 0.3718 - accuracy: 0.8395
Epoch 169/200
179/179 [==============================] - 0s 3ms/step - loss: 0.3713 - a

KerasClassifier(
	model=<function create_model at 0x000001ED3BC18820>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=200
	init=glorot_uniform
	class_weight=None
)

In [5]:
#predictions for one person
d = {
     'Pclass': [1],    # Ticket class -> 1 = 1st, 2 = 2nd, 3 = 3rd
     'Name': 'Leo',    # Doesn't matter 
     'Sex': 'male',    # male / female
     'Age': [20],      # in years 
     'SibSp': [1],     # Number of siblings / spouses aboard the Titanic
     'Parch': [0],     # Number of parents / children aboard the Titanic
     'Ticket':[0],     # Doesn't matter  
     'Fare': [57],     # 0 - 1000
     'Cabin': [0],     # Doesn't matter 
     'Embarked':'Q',   # Port of Embarkation -> C = Cherbourg, Q = Queenstown, S = Southampton
    }
# add columns that will be missing
if(d['Embarked'] == 'C'):
    d['Embarked_Q'] = [0]
    d['Embarked_S'] = [0]
if(d['Embarked'] == 'Q'):
    d['Embarked_C'] = [0]
    d['Embarked_S'] = [0]
if(d['Embarked'] == 'S'):
    d['Embarked_Q'] = [0]
    d['Embarked_C'] = [0]

df = pd.DataFrame(data=d)
df = clean_dataset(df) # prepare data

X_data = df.values.astype(float)

prediction = model_predictions.predict(X_data)
if(prediction):
    print("You survived!")
else:
    print("Better luck next time...")
print(prediction)

This port is most popular and replace all 'NA' to Q
1/1 [==============================] - 0s 211ms/step
Better luck next time...
[0]


In [6]:
#predictions using test dataset from kaggle
test_data = pd.read_csv("test.csv", index_col='PassengerId')

max_faret = test_data['Fare'].max()
min_faret = test_data['Fare'].min()
print(max_faret, min_faret)

test_data = clean_dataset(test_data) #prepare data to same format as train data
print(test_data.isnull().sum())  #one row has Fare NAN
test_data = test_data.fillna(10.000)
#print(test_data)
X_test = test_data.values.astype(float)
test_predictions = model_predictions.predict(X_test)
#print(test_predictions)
# create file to see the score on kaggle
submission = pd.DataFrame({
                'PassengerId': test_data.index,
                'Survived': test_predictions,
})
submission.sort_values('PassengerId', inplace=True)
submission.to_csv('submission.csv', index=False)

#prepare same dataframe as test for charts purposes
test_data['Survived'] = test_predictions

test_data.to_csv('test_data_results.csv', index = False)
test_data

512.3292 0.0
This port is most popular and replace all 'NA' to S
Pclass              0
Age                 0
Companions          0
Female              0
Male                0
Embarked_C          0
Embarked_Q          0
Embarked_S          0
Fare_(0, 50]        0
Fare_(50, 100]      0
Fare_(100, 200]     0
Fare_(200, 500]     0
Fare_(500, 1000]    0
dtype: int64
84/84 [==============================] - 0s 2ms/step


,Pclass,Age,Companions,Female,Male,Embarked_C,Embarked_Q,Embarked_S,"Fare_(0, 50]","Fare_(50, 100]","Fare_(100, 200]","Fare_(200, 500]","Fare_(500, 1000]",Survived
PassengerId,,,,,,,,,,,,,,
892,3,34.5,0,0,1,0,1,0,1,0,0,0,0,0
893,3,47.0,1,1,0,0,0,1,1,0,0,0,0,0
894,2,62.0,0,0,1,0,1,0,1,0,0,0,0,0
895,3,27.0,0,0,1,0,0,1,1,0,0,0,0,0
896,3,22.0,2,1,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,30.0,0,0,1,0,0,1,1,0,0,0,0,0
1306,1,39.0,0,1,0,1,0,0,0,0,1,0,0,1
1307,3,38.5,0,0,1,0,0,1,1,0,0,0,0,0
